In [1]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\musammil\Desktop\text-summarization\textenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\musammil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\musammil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [3]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
def get_sentence_embeddings(sentences):
  inputs = tokenizer(sentences, return_tensors='tf', truncation=True, max_length=512)

  outputs = model(inputs)

  token_embeddings = outputs.last_hidden_state

  attention_mask = tf.cast(tf.expand_dims(inputs['attention_mask'], -1), dtype=tf.float32)

  sum_embeddings = tf.reduce_sum(token_embeddings * attention_mask, axis=1)

  sum_mask = tf.reduce_sum(attention_mask, axis=1)

  mean_embeddings = sum_embeddings / sum_mask
  
  return mean_embeddings[0].numpy()

In [5]:
article = """
Artificial Intelligence (AI) is transforming industries across the world. From healthcare to finance, AI-powered solutions 
are being developed to automate processes, enhance decision-making, and improve efficiency. In healthcare, AI assists 
doctors by analyzing medical images, predicting diseases, and even suggesting treatment plans. Financial institutions 
use AI to detect fraudulent transactions and optimize trading strategies. Moreover, AI-driven chatbots and virtual 
assistants have revolutionized customer service, providing instant responses and personalized experiences. 
Despite these advancements, AI also raises ethical concerns, such as data privacy, bias in algorithms, and job displacement. 
As AI continues to evolve, it is crucial to ensure its responsible development and implementation to maximize benefits 
while minimizing risks.
"""

In [6]:
sentences = nltk.sent_tokenize(article)

In [7]:
sentence_embeddings = [get_sentence_embeddings(sentence) for sentence in sentences]

In [8]:
doc_embedding = np.mean(sentence_embeddings, axis=0).reshape(1, -1)

In [9]:
similarities = []
for emb in sentence_embeddings:
  sim = cosine_similarity(emb.reshape(1, -1), doc_embedding)[0][0]
  similarities.append(sim)

In [19]:
num_sentences = max(1, int(len(sentences) * 0.4))

In [20]:
sorted_indices = np.argsort(similarities)[::-1]
selected_indices = sorted(sorted_indices[:num_sentences])

In [21]:
summary = " ".join([sentences[idx] for idx in selected_indices])

print("Extractive Summary:")
print(summary)

Extractive Summary:
From healthcare to finance, AI-powered solutions 
are being developed to automate processes, enhance decision-making, and improve efficiency. Moreover, AI-driven chatbots and virtual 
assistants have revolutionized customer service, providing instant responses and personalized experiences.
